In [1]:
import os
import torch
import torch.nn as nn
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import csv
import torchvision.models as models
from sklearn.utils import shuffle
import warnings
import torch.nn.functional as F
warnings.filterwarnings("ignore")

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.get_device_name(0)

'GeForce GTX 1050 Ti'

In [2]:
# Hyperparameters
learning_rate = 0.0001
batch_size = 4

In [3]:
# Loading the data
df = pd.read_csv('D:/Sahil/Projects/Cancer Detection/dataset.csv', header=1)
df = shuffle(df)
df = df.to_numpy()

labels = np.zeros([len(df), 2])
for i,row in enumerate(df):
    if row[-1]==0:
        labels[i, 0] = 1
    else:
        labels[i, 1] = 1

train_data, test_data = df[:int(0.7*len(df)), :-1], df[int(0.7*len(df)):, :-1]
train_labels, test_labels = labels[:int(0.7*len(df))], labels[int(0.7*len(df)):]

print(test_labels.shape)

(66008, 2)


In [4]:
# Data Loader
class CancerDataset():

    def __init__(self, data, labels, root_dir, transform=None):
        self.dataframe = data
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, str(self.dataframe[idx, 0] + '.tif'))
        image = io.imread(img_name)
        image= transform.resize(image, (224, 224))
        sample = {'image': image, 'label': labels[idx]}
        
        if self.transform:
            sample = self.transform(sample)

        return sample

In [5]:
class ToTensor(object):
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        image = image.transpose((2,0,1))
        image = torch.from_numpy(image)
        label = torch.from_numpy(np.array([label]))
        label = torch.squeeze(label)
        return {'image': image.float(),
               'label': label.float()}


In [6]:
train_dataset = CancerDataset(data=train_data,labels=train_labels,
                                  root_dir="D:/Sahil/Projects/Cancer Detection/data/", transform=ToTensor())

test_dataset = CancerDataset(data=test_data, labels=test_labels,
                                  root_dir="D:/Sahil/Projects/Cancer Detection/data/", transform=ToTensor())



train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [7]:
model_conv=models.resnet18(pretrained=False)
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_conv.fc.parameters(), lr=0.005)

In [8]:
def train(epoch):
    model_conv.train()
    
    for i, sample in enumerate(train_loader):
        data, target = sample['image'].to(device), sample['label'].to(device)        
        optimizer.zero_grad()
        output = model_conv(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if(i%1000==0):
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i*len(data), len(train_loader.dataset),
                100. * i / len(train_loader), loss.item()))

In [9]:
def validate():
    model_conv.eval()
    correct = 0
    for i, sample in enumerate(test_loader):
        data, target = sample['image'].to(device), sample['label'].to(device) 
        output = model_conv(data)
        for j, row in enumerate(output):
            if torch.argmax(row).item() == torch.argmax(target[j]).item():
                correct += 1
                
    print("Validation loss: {}".format(correct/len(test_data)))

In [10]:
for i in range(3):
    train(i)
    with torch.no_grad():
        validate()
    

Train Epoch: 0 [0/154016 (0%)]	Loss: 0.750778
Train Epoch: 0 [4000/154016 (3%)]	Loss: 0.972124
Train Epoch: 0 [8000/154016 (5%)]	Loss: 0.677515
Train Epoch: 0 [12000/154016 (8%)]	Loss: 0.829308
Train Epoch: 0 [16000/154016 (10%)]	Loss: 0.555107
Train Epoch: 0 [20000/154016 (13%)]	Loss: 0.538508
Train Epoch: 0 [24000/154016 (16%)]	Loss: 0.535981
Train Epoch: 0 [28000/154016 (18%)]	Loss: 0.355882
Train Epoch: 0 [32000/154016 (21%)]	Loss: 0.677396
Train Epoch: 0 [36000/154016 (23%)]	Loss: 1.350936
Train Epoch: 0 [40000/154016 (26%)]	Loss: 0.568090
Train Epoch: 0 [44000/154016 (29%)]	Loss: 0.226026


KeyboardInterrupt: 